<h1><center><font size="6">Credit Card Fraud Detection Predictive Models</font></center></h1>
<h2><center><font size="5">Dealing with Imbalanced Data</font></center></h2>


<center><img src="https://images.unsplash.com/photo-1563013544-824ae1b704d3?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1050&q=80" width="600"></img></center>


# <a id='0'>Content</a>

- <a href='#1'>Introduction</a>  
- <a href='#2'>Import Dependencies</a>  
- <a href='#3'>Read the data</a>  
- <a href='#4'>Check the data</a>  
    - <a href='#41'>Glimpse the data</a>  
    - <a href='#410'>Profile Report</a>
    - <a href='#42'>Check missing data</a>
    - <a href='#420'>Check Duplicates</a>
    - <a href='#43'>Check data unbalance</a>
- <a href='#5'>Data exploration</a>
- <a href='#221'>Under-Sampling</a>
- <a href='#6'>Predictive models</a>  
    - <a href='#61'>RandomForrestClassifier</a> 
    - <a href='#62'>AdaBoostClassifier</a>
    - <a href='#64'>XGBoost</a>
    - <a href='#65'>Lazy Predict</a>
- <a href='#7'>Conclusions</a>
- <a href='#8'>References</a>


# <a id="1">Introduction</a>  

The datasets contains transactions made by credit cards in **September 2013** by european cardholders. This dataset presents transactions that occurred in two days, where we have **492 frauds** out of **284,807 transactions**. The dataset is **highly unbalanced**, the **positive class (frauds)** account for **0.172%** of all transactions.  

It contains only numerical input variables which are the result of a **PCA transformation**.   

Due to confidentiality issues, there are not provided the original features and more background information about the data.  

* Features **V1**, **V2**, ... **V28** are the **principal components** obtained with **PCA**;  
* The only features which have not been transformed with PCA are **Time** and **Amount**. Feature **Time** contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature **Amount** is the transaction Amount, this feature can be used for example-dependant cost-senstive learning.   
* Feature **Class** is the response variable and it takes value **1** in case of fraud and **0** otherwise.  



# <a id="2">Import Dependencies</a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pandas_profiling import profile_report

%matplotlib inline
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import gc
from datetime import datetime 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier
from sklearn import svm
import lightgbm as lgb
from lightgbm import LGBMClassifier
import xgboost as xgb

print('Setup Completed')

Setup Completed


# <a id='3'>Read the data</a> 

In [4]:
data_df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

# <a id='4'>Check the data</a> 

In [5]:
data_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

## <a id="41">Glimpse the data</a>

In [7]:
print("Credit Card Fraud Detection data -  rows:",data_df.shape[0]," columns:", data_df.shape[1])

Credit Card Fraud Detection data -  rows: 284807  columns: 31


## <a id="410">Profile Report</a>

function generates a comprehensive report that includes various statistics, insights, and visualizations about the DataFrame. This report can be helpful in understanding the data, identifying missing values, exploring data distributions, detecting correlations, and more.

In [ ]:
data_df.profile_report()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## <a id="42">Check missing data</a>  

Let's check if there is any missing data.

In [ ]:
total = data_df.isnull().sum().sort_values(ascending = False)
percent = (data_df.isnull().sum()/data_df.isnull().count()*100).sort_values(ascending = False)
pd.concat([total, percent], axis=1, keys=['Total', 'Percent']).transpose()

there is no missing data

## <a id="420">Check Duplicates</a>  

Let's check if there are any Duplicates in our data

In [ ]:
duplicated_values = data_df[data_df.duplicated()]
print(duplicated_values)

In [ ]:
data_df.drop_duplicates(inplace=True)

all done we droped the dupliccates

## <a id='43'>Check Unbalanced Data</a>

In [ ]:
data_df['Class'].value_counts()

In [ ]:
temp = data_df["Class"].value_counts()
df = pd.DataFrame({'Class': temp.index,'values': temp.values})

trace = go.Bar(
    x = df['Class'],y = df['values'],
    name="Credit Card Fraud Class - data unbalance (Not fraud = 0, Fraud = 1)",
    marker=dict(color="Red"),
    text=df['values']
)
data = [trace]
layout = dict(title = 'Credit Card Fraud Class - data unbalance (Not fraud = 0, Fraud = 1)',
          xaxis = dict(title = 'Class', showticklabels=True), 
          yaxis = dict(title = 'Number of transactions'),
          hovermode = 'closest',width=600
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='class')

Only 492 (or 0.172%) of transaction are fraudulent. That means the data is highly unbalanced with respect with target variable Class.

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,6))
s = sns.boxplot(ax = ax1, x="Class", y="Amount", hue="Class",data=data_df, palette="PRGn",showfliers=True)
s = sns.boxplot(ax = ax2, x="Class", y="Amount", hue="Class",data=data_df, palette="PRGn",showfliers=False)
plt.show();

In [ ]:
tmp = data_df[['Amount','Class']].copy()
class_0 = tmp.loc[tmp['Class'] == 0]['Amount']
class_1 = tmp.loc[tmp['Class'] == 1]['Amount']
class_0.describe()

In [ ]:
class_1.describe()

The real transaction have a larger mean value, larger Q1, smaller Q3 and Q4 and larger outliers; fraudulent transactions have a smaller Q1 and mean, larger Q4 and smaller outliers.

# <a id='221'>Under-Sampling</a>

Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions

Number of Fraudulent Transactions --> 492

In [ ]:
# Separete Data for Data Analysis
legit = data_df[data_df.Class == 0]
fraudulent = data_df[data_df.Class == 1]

In [ ]:
# Check Shape of our Labels
print(legit.shape)
print(fraudulent.shape)

In [ ]:
# Statistical measures of the data
legit.Amount.describe()

In [ ]:
fraudulent.Amount.describe()

In [ ]:
# Compare Both Transaction
data_df.groupby('Class').mean()

In [ ]:
legit_sample = legit.sample(n=492)

In [ ]:
# Concatenating two DataFrames
new_df = pd.concat([legit_sample, fraudulent], axis=0)

In [ ]:
# Get Equal Features in Our two labels is our Goal in Under-Sampling Technique
new_df['Class'].value_counts()

# <a id ='6'>Base line Modeling</a>

First we split our Data into two Classes

In [ ]:
X = new_df.drop(columns='Class', axis=1)
Y = new_df['Class']

Split your dataset into a training set and a test set:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

### <a id ='61'>RandomForest Classifier</a>

Let's run a model using the training set for training. Then, we will use the validation set for validation.

We will use as validation criterion GINI, which formula is GINI = 2 * (AUC) - 1, where AUC is the Receiver Operating Characteristic - Area Under Curve (ROC-AUC) [4]. Number of estimators is set to 100 and number of parallel jobs is set to 4.

We start by initializing the RandomForestClassifier.

In [ ]:
# Define your parameter values
RFC_METRIC = 'gini'  # or 'entropy' based on your choice
NUM_ESTIMATORS = 100  # or any desired number
NUM_JOBS = 4  # or any desired number
RANDOM_STATE = 42  # or any desired value

# Create the RandomForestClassifier with the defined parameter values
clf_rfc = RandomForestClassifier(
    n_jobs=NUM_JOBS,
    random_state=RANDOM_STATE,
    criterion=RFC_METRIC,
    n_estimators=NUM_ESTIMATORS,
    verbose=False
)

In [ ]:
clf_rfc.fit(X_train, y_train)

In [ ]:
y_pred = clf_rfc.predict(X_test)

### Feature Importance

we can classify if feature will be important to our model or no:

In [ ]:
target = 'Class'
predictors = ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',\
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19',\
       'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28',\
       'Amount']

In [ ]:
tmp = pd.DataFrame({'Feature': predictors, 'Feature importance': clf_rfc.feature_importances_})
tmp = tmp.sort_values(by='Feature importance',ascending=False)
plt.figure(figsize = (7,4))
plt.title('Features importance',fontsize=14)
s = sns.barplot(x='Feature',y='Feature importance',data=tmp)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.show()  

### Model Evaluation

Evaluate the model's performance using appropriate metrics:

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

confusion = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", confusion)

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)


### Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    # Add other hyperparameters to tune
}

grid_search = GridSearchCV(estimator=clf_rfc, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_clf = grid_search.best_estimator_

## <a id='64'>XGBoost Classifier</a>

In [ ]:
clf_XGB = xgb.XGBClassifier(
    learning_rate=0.1,  # Adjust as needed
    n_estimators=100,   # Adjust as needed
    max_depth=3,        # Adjust as needed
    random_state=42     # Set a random state for reproducibility
)

clf_XGB.fit(X_train, y_train)

In [ ]:
y_pred = clf_XGB.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

confusion = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", confusion)

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    # Add other hyperparameters to tune
}

grid_search = GridSearchCV(estimator=clf_XGB, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_clf = grid_search.best_estimator_

## <a id='62'>AdaBoostClassifier </a>

In [ ]:
# Create and train the AdaBoost classifier
clf_ada = AdaBoostClassifier(
    n_estimators=50,         # Adjust as needed
    learning_rate=1.0,       # Adjust as needed
    random_state=42           # Set a random state for reproducibility
)
clf_ada.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf_ada.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

confusion = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", confusion)

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

## <a id='65'>Lazy Predict</a>

Python library that provides a quick way to build and evaluate a wide range of machine learning models with minimal code. It's designed to help you get a sense of how different models perform on your dataset without the need for extensive manual configuration. It's particularly useful for initial exploratory data analysis and model selection.

install Lazy Predict

In [ ]:
# !pip install lazypredict

In [ ]:
from lazypredict.Supervised import LazyClassifier
# fitting data in LazyRegressor because
# here we are solving Regression use case.
clf = LazyClassifier(verbose=0,
                     ignore_warnings=False,
                     custom_metric=None)
  
# fitting data in LazyClassifier
models, predictions = clf.fit(X_train, X_test,
                              y_train, y_test)
# lets check which model did better
# on Breast Cancer Dataset
print(models)

# Save Model

In [ ]:
import joblib

joblib.dump(clf_rfc, 'best_model.pkl')  # Save the best model

# <a id='8'>Refrences</a>

[1] Credit Card Fraud Detection Database, Anonymized credit card transactions labeled as fraudulent or genuine, https://www.kaggle.com/mlg-ulb/creditcardfraud

[2] Principal Component Analysis, Wikipedia Page, https://en.wikipedia.org/wiki/Principal_component_analysis

[3] RandomForrestClassifier, http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

[4] ROC-AUC characteristic, https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve

[5] AdaBoostClassifier, http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html

[6] CatBoostClassifier, https://tech.yandex.com/catboost/doc/dg/concepts/python-reference_catboostclassifier-docpage/

[7] XGBoost Python API Reference, http://xgboost.readthedocs.io/en/latest/python/python_api.html

[8] LightGBM Python implementation, https://github.com/Microsoft/LightGBM/tree/master/python-package

[9] LightGBM algorithm, https://www.microsoft.com/en-us/research/wp-content/uploads/2017/11/lightgbm.pdf